In [1]:
import json
import speckle_fcns_v6 as fcns
import matplotlib.pyplot as plt
import numpy as np
from tkinter.filedialog import askdirectory

plt.close('all')

In [2]:
### user input ###
datapath = 'C:/data/BloodFlow/HHC'
scannerName = 'fessenden'
scanorder = 'zigzag' #'circle'

In [3]:
#########################################
###### do not modify below ##############
#########################################

In [4]:
calname = askdirectory(title='Select Calibration', initialdir=datapath)
scanname = askdirectory(title='Select Scan', initialdir=datapath) 

In [5]:
### data and algo parameters ###
if scanorder == 'zigzag':
    col = np.array([0, 0, 1, 1, 2, 2, 3, 3, 4, 4, 5, 5])
    row = np.array([0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1])
    theta = 3.14*np.array([-0.85, 0.85, -0.7, 0.7, -0.55, 0.55, -0.42, 0.42, -0.28, 0.28, -0.12, 0.12])
elif scanorder == 'circle':
    col = np.array([0, 1, 2, 3, 4, 5, 5, 4, 3, 2, 1, 0])
    row = np.array([0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1])
    theta = 3.14*np.array([-0.85, -0.7, -0.55, -0.42, -0.28, -0.12, 0.12, 0.28, 0.42, 0.55, 0.7, 0.85])

data_dict = { 
    'calname':  calname,
    'dataname': scanname, 
    'col': col, 
    'row': row,
    'theta': theta,
    'npositions': 12,  #must be even
    #'nstart': 0,
    #'nstop': 40,
    #'K': .12
    }
    
params_dict = {
    'n': 1.4,      # index of refraction
    'wv': np.array([8.5e-4]),    # wavelength, mm
    'mua': np.array([0.09]),   # absorption coefficient, mm^-1
    'musp': np.array([1.2]),     # reduced scattering coefficient, mm^-1
    'Db': 1e-6,    # diffusion coefficient of the scatterers, mm^2/s
    'dV2': 1,      # mean squared velocity of the scatterers, (mm/s)^2
    'Tmax': 1e-3,  # maximum exposure time being simulated, s
    'dtau': 1e-7,  # discretization of time step length, s
    'zb': 0.1,     # extrapolated boundary
    'cameras2use': np.array([0, 1, 2, 3]),
    'fitbeta': 0,
    'beta': 0.25
    #'dcontrast': np.array([0.4, 0, 0, 0]),
    #'betas': np.array([0.25, 0.25, 0.25, 0.5])
    }

In [6]:
### load json file ###
with open(data_dict.get('dataname') + '/scan_metadata.json') as json_file:
    json_dict = json.load(json_file) 
nrho   = json_dict.get('cameraParameters').get('numCameras')
scannerCameras = list(json_dict.get('cameraParameters').get('cameraInfo').keys())
for i in range(len(scannerCameras)):
    if scannerCameras[i] == scannerName:
        cids = list(json_dict.get('cameraParameters').get('cameraInfo').keys())[i+1:i+5]
gain = np.zeros((4,))
rho = np.zeros((4,))
for i in range(4):
    gain[i]=json_dict.get('cameraParameters').get('cameraInfo').get(cids[i]).get('gain') 
    rho[i]=json_dict.get('cameraParameters').get('cameraInfo').get(cids[i]).get('separation_mm')
data_dict['gain'] = gain
data_dict['rho'] = rho
params_dict['rho'] = rho
nt   = json_dict.get('cameraParameters').get('numImages')
exp = np.array([json_dict.get('delayParameters').get('pulseWidth_s')])
params_dict['exp']=exp
nexp = len(exp)
dt = 2/json_dict.get('cameraParameters').get('frameAcquisitionRate_Hz')

In [7]:
### load calibration data ###
caldata = fcns.read_bloodflow_protocsv2(data_dict.get('calname') + '/data.csv')
caldata = fcns.sortbycamera_proto2(caldata, cids, 2*nt)
caldata = fcns.removedark2(caldata)
caldata = fcns.averagetimeseries_proto(caldata)
params_dict['I0'] = caldata['mean_mean']

In [8]:
### load data ###
npositions = data_dict.get('npositions', 12)    
data = [{}] * npositions    
for i in range(npositions): 
    data[i] = fcns.read_bloodflow_protocsv2(data_dict.get('dataname') + '/location_' + str(i+1) + '.csv')
    data[i] = fcns.sortbycamera_proto2(data[i], cids, 2*nt)
    data[i] = fcns.removedark2(data[i])
    if ('nstart' in data_dict.keys()) or ('nstop' in data_dict.keys()):
        data[i] = fcns.croptime_proto(data[i], data_dict)
        print('cropping')
    data[i] = fcns.getpulse(data[i], data_dict, dt)

In [9]:
### average data ###
for i in range(npositions):
    data[i]=fcns.averagetimeseries_proto(data[i])
    data[i]=fcns.getridofzeros(data[i])

In [10]:
### plot data ###
%matplotlib qt
fcns.plotscandata(data, data_dict, 'pcontrast')
fcns.showResult(data, data_dict, 'pulse')
fcns.plotscandataintime(data, data_dict)    
fcns.plotscandata(data, data_dict, 'mean')
fcns.plotscandata(data, data_dict, 'std')
fcns.plotscandata(data, data_dict, 'contrast')
fcns.showAverageData(data, data_dict, 'mean_mean')
fcns.showAverageData(data, data_dict, 'std_mean')
fcns.showAverageData(data, data_dict, 'contrast_mean')

In [11]:
################################
##### fit for parameter(s) #####
################################

In [12]:
### attenuation ###
### define independent variables
c2u = params_dict.get('cameras2use')        
p = np.squeeze(params_dict['rho'])[c2u]
pp = np.linspace(np.amin(p), np.amax(p))
pp2 = np.linspace(0, np.amax(p))
exp = params_dict['exp']
expp = np.array([params_dict['exp'][0]])

### calibrate intensity ###
ci = fcns.getIntensityCalibration(params_dict)
for i in range(npositions):
    data[i]['mean'] *= ci
    data[i]['mean_mean'] *= ci
    data[i]['std'] *= ci
    data[i]['std_mean'] *= ci

### find mu_eff & mua ###
mean = np.zeros((npositions, len(c2u), 1))
std = np.zeros((npositions, len(c2u), 1))
for i in range(npositions):
    mean[i, :, 0]  = data[i]['mean_mean'][c2u]
    std[i, :, 0] = data[i]['std_mean'][c2u]
mu_eff, mua   = fcns.fitattenuation(mean,  p, pp, params_dict['musp'], std)
for i in range(npositions):
    data[i]['mu_eff'] = mu_eff[i]
    data[i]['mua'] = mua[i]
fcns.showResult(data, data_dict, 'mu_eff')
fcns.showResult(data, data_dict, 'mua')

In [13]:
### flow ###
### define independent variables
params_dict['C0']=data[0]['contrast_mean'] # relative BFI only
params_dict['mua']=data[0]['mua']
params_dict['cameras2use'] = np.array([1, 2, 3]) #punting on 35 mm camera for now
c2u = params_dict.get('cameras2use')
p = np.squeeze(params_dict['rho'])[c2u]
pp = np.linspace(np.amin(p), np.amax(p))
pp2 = np.linspace(0, np.amax(p))

### calibrate contrast ###
cc = fcns.getContrastCalibration(params_dict) 
for i in range(npositions):   
    data[i]['contrast'] *= cc
    data[i]['contrast_mean'] *= cc

### find alpha*Db ###
contrast = np.zeros((npositions, len(c2u), 1))
for i in range(npositions):
    contrast[i, :, 0] = data[i]['contrast_mean'][c2u]
alphaDb, beta, offset    = fcns.fitflow_multi(params_dict, contrast,  p, pp2, exp, expp, mua)  #using 1st mua
for i in range(npositions):
    data[i]['rCBF'] = alphaDb[i]
fcns.showResult(data, data_dict, 'rCBF') 